#### Step 1 - Ingest Nested JSON Data

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
# Let's import some packages
from pyspark.sql.types import *
from pyspark.sql.functions import *

name_schema = StructType(fields = [StructField("forename",StringType(),True),
                                            StructField("surname",StringType(),True)
                                  ])

In [0]:
driver_schema = StructType(fields = [
    StructField("driverId", IntegerType(),False),
    StructField("driverRef", StringType(),True),
    StructField("number", IntegerType(),True),
    StructField("code", StringType(),True),
    StructField("name", name_schema,True),
    StructField("dob", DateType(),True),
    StructField("nationality", StringType(),True),
    StructField("url", StringType(),True)    
])

In [0]:
drivers_df = spark.read \
.schema(driver_schema) \
.json(f"{raw_folder_path}/drivers.json")

#### Step 2 - Rename columns and add new ones

In [0]:
drivers_renamed_df = add_ingestion_date(drivers_df).withColumnRenamed("driverId","driver_id")\
.withColumnRenamed("dirverRef","driver_ref")\
.withColumn("name", concat(col("name.forename"),lit(" "), col("name.surname")))

#### Step 3 - Drop unwanted columns

In [0]:
drivers_dropped_df = drivers_renamed_df.drop("url")

#### Step 4 - Write output to processed container in parquet format

In [0]:
drivers_dropped_df.write.mode("overwrite").parquet(f"{processed_folder_path}/drivers")

In [0]:
drivers_dropped_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.driversSQL")

#### Step 6 - Send exit statement for any dbutils.notebook.run cells

In [0]:
dbutils.notebook.exit("Success")

Success